In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatText
from IPython.display import clear_output
import seaborn as sns
from scipy.integrate import solve_ivp


In [4]:

def harmonic_potential(x, k=1.0):
    potential = 0.5 * k * x**2
    force = -k * x
    return potential, force

def force_function(x, k, a):
    force = -2 * k * x * (x**2 - a)
    return force

def double_force_potential_function(x, k, a):
    potential = 0.5 * k * (x**2 - a)**2
    return potential

def dual_harmonic_potential(x, k=1.0, a = 1.0):
    potential = double_force_potential_function(x,k,a)
    force = force_function(x, k,a)
    return potential, force

def smoothed_positive_potential(x, k=1.0, a=1.0, b=0.01):
    base_potential = 0.5 * k * (x**2 - a)**2
    
    # Gaussian peaks centered at ±sqrt(b)
    gaussian_1 = k * np.exp(-(x - np.sqrt(a))**2 / (2 * b**2))
    gaussian_2 = k * np.exp(-(x + np.sqrt(a))**2 / (2 * b**2))
    
    # Combined potential and force
    potential = base_potential + gaussian_1 + gaussian_2

    force_base = -2 * k * x * (x**2 - a)
    force_g1 = k * (x - np.sqrt(a)) / b**2 * np.exp(-(x - np.sqrt(a))**2 / (2 * b**2))
    force_g2 = k * (x + np.sqrt(a)) / b**2 * np.exp(-(x + np.sqrt(a))**2 / (2 * b**2))
    force = force_base - force_g1 - force_g2
    
    return potential, force

def simulate_motion(potential_func, params, x0=1.0, v0=0.0, t_min=0, t_max=36, dt=0.01):
    def dynamics(t, y):
        x, v = y
        _, force = potential_func(x, **params)
        return [v, force]

    t_span = (t_min, t_max)
    t_eval = np.arange(t_min, t_max, dt)
    sol = solve_ivp(dynamics, t_span, [x0, v0], t_eval=t_eval)
    return sol.t, sol.y[0], sol.y[1]



def plot_potential_phase_space(potential_case="Harmonic", k=1.0,a = 1.0, b=0.01):
    clear_output(wait=True)
    x = np.linspace(-50, 50, 500)
    

    if potential_case == "Harmonic":
        potential, _ = harmonic_potential(x, k)
        params = {'k': k}
        potential_func = harmonic_potential
    elif potential_case=="Smooth Harmonic":
        potential, _ = smoothed_positive_potential(x, k, a, b)
        params = {'k': k, 'a': a, 'b' : b}
        potential_func = smoothed_positive_potential
    elif potential_case== "Dual Harmonic":
        potential, _ = dual_harmonic_potential(x, k, a)
        params = {'k': k, 'a': a}
        potential_func = dual_harmonic_potential    

    t, x_vals, v_vals = simulate_motion(potential_func, params)

    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    
    ax[0].plot(x, np.log(1 + potential), label="Potential")
    ax[0].set_xlabel("x")
    ax[0].set_ylabel("Potential")
    ax[0].set_xlim(-10,10)
    ax[0].set_title(f"{potential_case} Potential")
    ax[0].grid(True)
    ax[0].legend()

    sns.kdeplot(x=x_vals, y=v_vals,  cmap="viridis", fill=True, ax=ax[1])
    ax[1].set_xlabel("x")
    ax[1].set_ylabel("v")
    ax[1].set_title("Phase Space")
    ax[1].grid(True)
    ax[1].legend()




In [5]:
interact(
    plot_potential_phase_space,
    potential_case=["Harmonic","Dual Harmonic", "Smooth Harmonic"],
    k=FloatText(value=10.0, description="k (Harmonic)"),
    a=FloatText(value=20.0, description="a (Dual Hump)"),
    b=FloatText(value=2, description="b (Smooth)"),
    
)

interactive(children=(Dropdown(description='potential_case', options=('Harmonic', 'Dual Harmonic', 'Smooth Har…

<function __main__.plot_potential_phase_space(potential_case='Harmonic', k=1.0, a=1.0, b=0.01)>